In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import os

# Specify the path to the existing database file
db_path = r"C:\Users\lasra\Desktop\Illinos Tech Courses\CS727-Relational Database Design\DEL 5 - 2\D5\Final - 02\womenwordcup2023_module5.db"

# Check if the database file exists
if not os.path.exists(db_path):
    messagebox.showerror("Error", f"Database file {db_path} does not exist. Please check the path.")
    exit(1)

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
c = conn.cursor()

# GUI Application
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Database Manager")

        self.create_widgets()

    def create_widgets(self):
        # Frame for buttons
        btn_frame = tk.Frame(self.root)
        btn_frame.pack(pady=10)

        self.query1_btn = tk.Button(btn_frame, text="Rank Rounds by Attendance", command=self.rank_rounds_by_attendance)
        self.query1_btn.grid(row=0, column=0, padx=5)

        self.query2_btn = tk.Button(btn_frame, text="Rank Locations by Attendance", command=self.rank_locations_by_attendance)
        self.query2_btn.grid(row=1, column=0, padx=5)

        self.query3_btn = tk.Button(btn_frame, text="Show Top Teams by Wins", command=self.show_top_teams_by_wins)
        self.query3_btn.grid(row=2, column=0, padx=5)

        self.query4_btn = tk.Button(btn_frame, text="Result and Location of Higher Than Average Attendance Games", command=self.result_and_location_higher_than_average_attendance_games)
        self.query4_btn.grid(row=3, column=0, padx=5)

        self.query5_btn = tk.Button(btn_frame, text="Players who have scored at the Games with Higher than Average Attendance Games", command=self.players_scored_higher_than_average_attendance_games)
        self.query5_btn.grid(row=4, column=0, padx=5)

        # Frame for output
        self.output_frame = tk.Frame(self.root)
        self.output_frame.pack(pady=10)

    def run_query(self, query, column_titles):
        self.clear_output_frame()
        try:
            c.execute(query)
            records = c.fetchall()
            # Display column titles
            title_row = tk.Frame(self.output_frame)
            title_row.pack()
            for title in column_titles:
                title_label = tk.Label(title_row, text=title, borderwidth=1, relief="solid", width=20)
                title_label.pack(side="left")
            # Display query results
            for record in records:
                data_row = tk.Frame(self.output_frame)
                data_row.pack()
                for item in record:
                    data_label = tk.Label(data_row, text=item, borderwidth=1, relief="solid", width=20)
                    data_label.pack(side="left")
        except sqlite3.Error as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def clear_output_frame(self):
        for widget in self.output_frame.winfo_children():
            widget.destroy()

    def rank_rounds_by_attendance(self):
        self.run_query("""
            SELECT
                RANK() OVER (
                    ORDER BY
                        SUM(g.Attendance) DESC
                ) AS Round_Rank,
                r.Round_Name,
                SUM(g.Attendance) AS Total_Attendance
            FROM Score s
            JOIN Round r ON r.RoundID = s.RoundID
            JOIN Game g ON g.MatchID = s.MatchID
            GROUP BY
                r.RoundID,
                r.Round_Name
            ORDER BY Round_Rank
        """, ("Round Rank", "Round Name", "Total Attendance"))

    def rank_locations_by_attendance(self):
        self.run_query("""
            SELECT
                RANK() OVER (
                    ORDER BY
                        SUM(g.Attendance) DESC
                ) AS Location_Rank,
                l.Location_Name,
                SUM(g.Attendance) AS Total_Attendance
            FROM Score s
            JOIN Location l ON l.LocationID = s.RoundID
            JOIN Game g ON g.MatchID = s.MatchID
            GROUP BY
                l.LocationID,
                l.Location_Name
            ORDER BY Location_Rank
        """, ("Location Rank", "Location Name", "Total Attendance"))

    def show_top_teams_by_wins(self):
        self.run_query("""
            SELECT 
                Team_Name,
                Win AS Number_Of_Wins,
                NTILE(3) OVER (
                    ORDER BY Win DESC
                ) AS Wins_Tile
            FROM Team
        """, ("Team Name", "Number Of Wins", "Wins Tile"))

    def result_and_location_higher_than_average_attendance_games(self):
        self.run_query("""
            SELECT
                g.Match_Date,
                g.Result,
                g.Attendance,
                l.Location_Name
            FROM Score s
            JOIN Game g ON g.MatchID = s.MatchID
            JOIN Location l ON s.LocationID = l.LocationID
            WHERE g.Attendance > (
                    SELECT AVG(Attendance)
                    FROM Game
                )
            GROUP BY
                g.Match_Date,
                g.Result,
                g.Attendance,
                l.Location_Name
            ORDER BY g.Match_Date DESC
        """, ("Match Date", "Result", "Attendance", "Location Name"))

    def players_scored_higher_than_average_attendance_games(self):
        self.run_query("""
            SELECT
                p.Player_First_Name,
                p.Player_Last_Name,
                s.Goal_Count,
                g.Match_Date,
                g.Attendance 
            FROM Score s
                JOIN Game g ON g.MatchID = s.MatchID
                JOIN Player p ON s.PlayerID = p.PlayerID
            WHERE g.Attendance > (
                    SELECT AVG(Attendance)
                    FROM Game
                ) AND s.Goal_Count > 0
            ORDER BY g.Match_Date DESC
        """, ("Player First Name", "Player Last Name", "Goal Count", "Match Date", "Attendance"))


def main():
    root = tk.Tk()
    app = App(root)
    root.mainloop()

if __name__ == "__main__":
    main()

# Close the connection when the application exits
conn.close()
